In [2]:
lines = []
lines.append("DGRP population parameters")

In [3]:
lines.append("")
lines.append("Ungapped size of r5 assembly, source: https://www.ncbi.nlm.nih.gov/assembly/GCF_000001215.2/")
total_sites = 136_826_056
lines.append(f"Total sites: {total_sites}")

In [4]:
pi_values = []
with open(snakemake.input["heterozygosity"], 'r') as f:
    _ = f.readline() # skip header
    for line in f:
        pi_values.append(float(line.strip().split()[-1]))

In [5]:
seg_sites = len(pi_values)
avg_pi_per_site = sum(pi_values)/total_sites
S_per_site = seg_sites/total_sites

lines.append("")
lines.append("In the imputed DGRP VCF:")
lines.append(f"There are {seg_sites} segregating sites.")
lines.append(f"Average pi per site is {avg_pi_per_site:.6f}.")
lines.append(f"S per site is {S_per_site:.6f}.")
lines.append("For comparison, we can relate our parameters to Garud et al. (2015) (https://doi.org/10.1371/journal.pgen.1005004):")
lines.append('"S and pi estimates in DGRP short intron data were measured to be 5.8% and 1.2% per bp, respectively."')

In [6]:
mutation_rate = 1.39e-9
recombination_rate = 1e-8
mu_over_r = mutation_rate/recombination_rate
lines.append("")
lines.append("The original population parameters taken from Arguello et al. (2019) (https://doi.org/10.1093/gbe/evz022):")
lines.append(f"Mutation rate: {mutation_rate}")
lines.append(f"Recombination rate: {recombination_rate}")
lines.append(f"Mu over r: {mu_over_r:6f}")

In [7]:
# empirical_pi =  0.0045
empirical_pi =  avg_pi_per_site

lines.append("")
lines.append("Empirical_pi is approximately 2*hap_population_size*mutation_rate in case of Wright-Fisher.")
hap_population_size = empirical_pi/(2*mutation_rate)
lines.append(f'Estimated 2Ne: {hap_population_size:,.0f}')

In [8]:
lines.append("")
desired_hap_Ne = 100_000
lines.append(f'To rescale so 2Ne = {desired_hap_Ne:,.0f}:')
new_mutation = empirical_pi/(2*desired_hap_Ne)
new_recombination = new_mutation/mu_over_r
lines.append(f'Rescaled mutation rate: {new_mutation:.4g}\nRescaled recombination rate: {new_recombination:.4g}')

In [9]:
lines.append("")
lines.append(
    "The extent of a sweep signature is roughly d = 0.1 * (s/r). Since we have our rescaled recombination rate, to see which selection coefficient we need to simulate, we can solve for s = (rd)/0.1 for our minimum and maximum desired sweep signature."
)

In [10]:
min_signature = 1000    # 1 kb
max_signature = 1000000 # 1 Mb
min_s = (new_recombination*min_signature)/0.1
max_s = (new_recombination*max_signature)/0.1

lines.append(f'To see a sweep signature of {min_signature}bp, s={min_s:6f}.')
lines.append(f'To see a sweep signature of {max_signature}bp, s={max_s:6f}.')

In [11]:
with open(snakemake.output[0], 'w') as f:
    f.write("\n".join(lines))